In [1]:
%run PATTERN_OBSERVER.ipynb

In [2]:
from ipywidgets import VBox,HBox,Text,DatePicker,Button,Layout,Dropdown
from ipywidgets import Color # pretty color for apply button
from pathlib import Path # for parsing directory contents

In [48]:
'''
backtest location
date range
skip dates - TODO
return compute type (cash,future,both)
'''
class SETTINGS(ISubject):
    ####################################
    # static
    ####################################
    _default_backtest_path=r'C:\Users\ahkar\OneDrive\Documents\Data\B3'
    
    _return_types=[
        'Cash + Future',
        'Cash',
        'Future',
    ]
    
    '''each entry of SETTINGS._option_names needs to be a widget and also instantiable'''
    _option_names=[
        'backtest_path',
        'date_from',
        'date_to',
        'return_type',
    ]

    ####################################
    # constructor
    ####################################
    def __init__(self,
        name='SETTINGS',
        width='25%'
        ):
        self._width=width
        
        self.name=name
        self.widget=self._make()
        
        self.applied_settings=self._current_settings # update checkpoint
        self._update_reference() # update reference
        self._observers=set() # for observer pattern
        
    ####################################
    # observer pattern
    ####################################
    # subject
    def attach(self,observer : IObserver) -> None:
        print('OBSERVER PATTERN',':',observer.name,'OBSERVES',self.name)
        self._observers.add(observer)
        
    def detach(self,observer : IObserver) -> None:
        print('OBSERVER PATTERN',':',observer.name,'STOPS OBSERVING',self.name)
        self._observers.remove(observer)
        
    def notify(self) -> None:
        print('OBSERVER PATTERN',':',self.name,'NOTIFIES',len(self._observers),'OBSERVERS')
        
        # print receivers
        for observer in self._observers:
            print('OBSERVER PATTERN',':',self.name,'NOTIFIES',observer.name)
            observer.react(self.name,self.reference)
            
    ####################################
    # buttons
    ####################################
    def _button_inactive(self,button):
        button.style.button_color=Color(None).name
        button.disabled=True
        
    def _button_active(self,button,color='pink'):
        button.style.button_color=color
        button.disabled=False
        
    def _update_button_state(self):
        if self.applied_settings==self._current_settings:
            # if nothing changed
            self._button_inactive(self._control_widgets[0])
            self._button_inactive(self._control_widgets[1])
        else:
            # if something changed
            self._button_active(self._control_widgets[0],color='lightgreen')
            self._button_active(self._control_widgets[1],color='pink')

    def _click_apply(self,change):
        print(self.name,':','CLICKED',change.description)
        self.applied_settings=self._current_settings # update checkpoint
        self._update_button_state() # update button state
        self._update_reference() # update reference
        self.notify() # notify observers

    def _click_cancel(self,change):
        print(self.name,':','CLICKED',change.description)

        # reset to last applied value
        for widget,value in zip(self._option_widgets,self.applied_settings):
            widget.value=value

        self._update_button_state() # update button state

    def _click_reset(self,change):
        print(self.name,':','CLICKED',change.description)
        self._option_widgets[1].value=None
        self._option_widgets[2].value=None
        self._option_widgets[3].value=SETTINGS._return_types[0]
        self._update_button_state() # update button state

    def _click_refresh(self,change):
        print(self.name,':','CLICKED',change.description)
        # refresh is mostly same as apply
        # difference is no apply and button color change
        self._update_reference() # update reference
        self.notify() # notify observers

    def _onchange_backtest_path(self,change):
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state
        
    def _onchange_date_from(self,change):
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state

    def _onchange_date_to(self,change):
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state
        
    def _onchange_return_type(self,change):
        print(self.name,':','CHANGED',change.owner.description,change.new)
        self._update_button_state() # update button state

    def _make(self):
        # pretty controls
        controls=HBox(self._make_controls())
        
        # pretty checkboxes
        options=VBox(
            children=self._make_options(),
            layout=Layout(height='300px')
        )
        
        # return
        return VBox(
            children=[controls,options],
            layout=Layout(border='1px solid black',width=self._width,overflow_x='scroll')
        )
    
    def _make_controls(self):
        # checkbox controls
        button_apply=Button(description='apply',disabled=True)
        button_apply.on_click(self._click_apply)

        button_cancel=Button(description='cancel',disabled=True)
        button_cancel.on_click(self._click_cancel)

        button_reset=Button(description='reset')
        button_reset.on_click(self._click_reset)

        button_refresh=Button(description='refresh')
        button_refresh.on_click(self._click_refresh)

        return [
            button_apply,
            button_cancel,
            button_reset,
            button_refresh,
        ]

    def _make_options(self):
        # backtest_path
        backtest_path=Text(
            description='path',
            value=SETTINGS._default_backtest_path,
            placeholder='what is a placeholder?',
            disabled=False,
            #continuous_update=False,
        )
        backtest_path.observe(self._onchange_backtest_path,names='value')
        
        # date_from
        date_from=DatePicker(description='date from')
        date_from.observe(self._onchange_date_from,names='value')
        
        # date_to
        date_to=DatePicker(description='date to')
        date_to.observe(self._onchange_date_to,names='value')
            
        # return_type
        return_type=Dropdown(description='return type',options=SETTINGS._return_types)
        return_type.observe(self._onchange_return_type,names='value')
        
        #instantiate and return list of widgets / options
        return list(map(eval,SETTINGS._option_names))

    @property
    def _control_widgets(self):
        return self.widget.children[0].children
    
    @property
    def _option_widgets(self):
        return self.widget.children[1].children
    
    @property
    def _current_settings(self):
        return [x.value for x in self._option_widgets]
    
    '''get applied setting by name'''
    def _applied_setting_by_name(self,name):
        return self.applied_settings[SETTINGS._option_names.index(name)]

    '''return list of contents in backtest_path'''
    def _refresh_all_backtests(self):
        p=self._applied_setting_by_name('backtest_path')
        return {x.stem:x for x in Path(p).iterdir()}
        
    '''return list of books in backtest_path'''
    def _refresh_all_books(self):
        # TODO update this
        return {x:x for x in ['Trading','Quote','MM2','Hedge','Hit']}

    '''
    # TODO this assumes settings are correct, backtest_path can be wrong!
    '''
    def _update_reference(self):
        self.reference={
            'all_backtests':self._refresh_all_backtests(),
            'all_books':self._refresh_all_books(),
            'backtest_path':self._applied_setting_by_name('backtest_path'),
            'date_from':self._applied_setting_by_name('date_from'),
            'date_to':self._applied_setting_by_name('date_to'),
            'return_type':self._applied_setting_by_name('return_type'),
        }

        # show internals
        self._logging()
        
    def _logging(self):
        for k,v in self.reference.items():
            print(self.name,':','REFERENCE',':',k,':',v)
        
# example
# SETTINGS().widget

In [38]:
'''
a=SETTINGS()
a.widget
a.reference['all_backtests'][0].parent
'''
None